In [2]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

In [3]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')

def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)

                X.append(lines)
                y.append(path)

    return X, y

train_path = os.path.join(dir_path, r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\Train\new train')
X_data, y_data = get_data(train_path)


100%|██████████| 4/4 [00:12<00:00,  3.09s/it]


In [5]:
import pickle

pickle.dump(X_data, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_data.pkl', 'wb'))
pickle.dump(y_data, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_data.pkl', 'wb'))


In [6]:
test_path = os.path.join(dir_path, r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\Test\new test')
X_test, y_test = get_data(test_path)

pickle.dump(X_test, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_test.pkl', 'wb'))
pickle.dump(y_test, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_test.pkl', 'wb'))


100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


In [7]:
import pickle

X_data = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_data.pkl', 'rb'))
y_data = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_data.pkl', 'rb'))

X_test = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_test.pkl', 'rb'))
y_test = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_test.pkl', 'rb'))


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=400)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


In [12]:
from gensim.models import FastText
import os

# Đường dẫn đến file .bin
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
fasttext_model_path = os.path.join(dir_path, r"E:\hoc hanh\THUY LOI HOC\NLP\btl1\cc.vi.300.bin\cc.vi.300.bin")

# Tải mô hình FastText
model = FastText.load_fasttext_format(fasttext_model_path)

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in model.wv.key_to_index:
                sentence.append(model.wv[word])
        word2vec_data.append(sentence)
    return word2vec_data

# Giả sử X_data và X_test là dữ liệu của bạn
X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)


C:\Users\COMPUTER\AppData\Local\Temp\ipykernel_20660\3068195609.py:9: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format(fasttext_model_path)


In [13]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ 


array(['Gioi tinh', 'Hackers va Virus', 'The gioi tre', 'Thoi trang'],
      dtype='<U16')

In [37]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Chuyển đổi dữ liệu đầu vào thành dạng ma trận
max_len = 100 # Độ dài tối đa của mỗi câu
X_data_pad = pad_sequences(X_data_w2v, maxlen=max_len, dtype='float32')
X_test_pad = pad_sequences(X_test_w2v, maxlen=max_len, dtype='float32')

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, 300))) # 300 là số chiều của embedding vector
model.add(Dense(64, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax')) # Số lớp đầu ra

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_data_pad, y_data_n, batch_size=32, epochs=15, validation_data=(X_test_pad, y_test_n))

# Dự đoán xác suất cho mỗi lớp trên tập test
y_pred_prob = model.predict(X_test_pad)

# Lấy ra lớp có xác suất cao nhất cho mỗi mẫu
y_pred = np.argmax(y_pred_prob, axis=1)

# In báo cáo phân loại
print(classification_report(y_test_n, y_pred, target_names=encoder.classes_))


Epoch 1/15
46/46 [==============================] - 6s 85ms/step - loss: 1.2038 - accuracy: 0.4462 - val_loss: 0.9567 - val_accuracy: 0.4807
Epoch 2/15
46/46 [==============================] - 3s 65ms/step - loss: 0.8164 - accuracy: 0.6246 - val_loss: 0.7694 - val_accuracy: 0.7612
Epoch 3/15
46/46 [==============================] - 3s 65ms/step - loss: 0.6036 - accuracy: 0.7641 - val_loss: 0.9181 - val_accuracy: 0.7006
Epoch 4/15
46/46 [==============================] - 3s 65ms/step - loss: 0.7983 - accuracy: 0.6815 - val_loss: 0.6594 - val_accuracy: 0.7029
Epoch 5/15
46/46 [==============================] - 3s 76ms/step - loss: 0.4876 - accuracy: 0.8085 - val_loss: 0.5610 - val_accuracy: 0.8085
Epoch 6/15
46/46 [==============================] - 3s 67ms/step - loss: 0.3877 - accuracy: 0.8452 - val_loss: 0.3789 - val_accuracy: 0.8503
Epoch 7/15
46/46 [==============================] - 3s 65ms/step - loss: 0.3049 - accuracy: 0.8924 - val_loss: 0.3634 - val_accuracy: 0.8857
Epoch 8/15
46

In [38]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Chuyển đổi dữ liệu đầu vào thành dạng ma trận
max_len = 100 # Độ dài tối đa của mỗi câu
X_data_pad = pad_sequences(X_data_w2v, maxlen=max_len, dtype='float32')
X_test_pad = pad_sequences(X_test_w2v, maxlen=max_len, dtype='float32')

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, 300))) # 300 là số chiều của embedding vector
model.add(Dense(64, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax')) # Số lớp đầu ra

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_data_pad, y_data_n, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test_n))

# Dự đoán xác suất cho mỗi lớp trên tập test
y_pred_prob = model.predict(X_test_pad)

# Lấy ra lớp có xác suất cao nhất cho mỗi mẫu
y_pred = np.argmax(y_pred_prob, axis=1)

# In báo cáo phân loại
print(classification_report(y_test_n, y_pred, target_names=encoder.classes_))


Epoch 1/10
46/46 [==============================] - 5s 72ms/step - loss: 1.1240 - accuracy: 0.5156 - val_loss: 0.9203 - val_accuracy: 0.6501
Epoch 2/10
46/46 [==============================] - 3s 69ms/step - loss: 0.7295 - accuracy: 0.6891 - val_loss: 0.5958 - val_accuracy: 0.7084
Epoch 3/10
46/46 [==============================] - 3s 73ms/step - loss: 0.6359 - accuracy: 0.7606 - val_loss: 0.4927 - val_accuracy: 0.8117
Epoch 4/10
46/46 [==============================] - 4s 86ms/step - loss: 0.4394 - accuracy: 0.8341 - val_loss: 0.3946 - val_accuracy: 0.8526
Epoch 5/10
46/46 [==============================] - 4s 97ms/step - loss: 0.3757 - accuracy: 0.8668 - val_loss: 0.6262 - val_accuracy: 0.7006
Epoch 6/10
46/46 [==============================] - 3s 70ms/step - loss: 0.3250 - accuracy: 0.8869 - val_loss: 0.3553 - val_accuracy: 0.9046
Epoch 7/10
46/46 [==============================] - 3s 69ms/step - loss: 0.2695 - accuracy: 0.9126 - val_loss: 0.2596 - val_accuracy: 0.9165
Epoch 8/10
46